In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from tabulate import tabulate

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [13]:
site_A_dataset=pd.read_csv("/content/drive/MyDrive/flextreck_datasets/site_A_data.csv",parse_dates=["Timestamp_Local"],index_col=["Timestamp_Local"])
site_B_dataset=pd.read_csv("/content/drive/MyDrive/flextreck_datasets/site_B_data.csv",parse_dates=["Timestamp_Local"],index_col=["Timestamp_Local"])
site_C_dataset=pd.read_csv("/content/drive/MyDrive/flextreck_datasets/site_C_data.csv",parse_dates=["Timestamp_Local"],index_col=["Timestamp_Local"])
site_D_dataset=pd.read_csv("/content/drive/MyDrive/flextreck_datasets/site_D_data.csv",parse_dates=["Timestamp_Local"],index_col=["Timestamp_Local"])
site_E_dataset=pd.read_csv("/content/drive/MyDrive/flextreck_datasets/site_E_data.csv",parse_dates=["Timestamp_Local"],index_col=["Timestamp_Local"])
site_F_dataset=pd.read_csv("/content/drive/MyDrive/flextreck_datasets/site_F_data.csv",parse_dates=["Timestamp_Local"],index_col=["Timestamp_Local"])

In [36]:
def pretty_print(df, title):
      print(f"\n=== {title} ===")
      print(tabulate(df.reset_index(), headers='keys', tablefmt='psql', showindex=False))

In [4]:
def get_season_au(month):
    if month in [12, 1, 2]:
        return 'Summer'
    elif month in [3, 4, 5]:
        return 'Autumn'
    elif month in [6, 7, 8]:
        return 'Winter'
    else:
        return 'Spring'

In [5]:
def make_time_numeric(df):
  df['Hour'] = df.index.hour
  df['Weekday'] = df.index.weekday  # 0 = Monday, 6 = Sunday
  df['Is_Weekend'] = df['Weekday'].isin([5,6])  # Saturday/Sunday
  df['Date'] = df.index.date  # for daily aggregation
  df['Month'] = df.index.month
  return df


In [54]:
def flag_analysis(df,df_name):
  print(f'{df_name} FLAG ANALYSİS')

  df=make_time_numeric(df)
  df['Season'] = df['Month'].apply(get_season_au)
  flag_counts_day = (
        df.groupby(df.index.date)['Demand_Response_Flag']
        .value_counts()
        .unstack(fill_value=0)
    )
    # group by month
  flag_counts_month = (
      df.groupby(pd.Grouper(freq='M'))['Demand_Response_Flag']
        .value_counts()
        .unstack(fill_value=0)
  )
  # group by season
  flag_counts_season = (
      df.groupby('Season')['Demand_Response_Flag']
        .value_counts()
        .unstack(fill_value=0)
  )

  # keep only flag activity
  flag_activity = df[df['Demand_Response_Flag'].isin([-1, 1])].copy()

  # extract just the time part (HH:MM)
  flag_activity['time_only'] = flag_activity.index.time

  # count occurrences of each time
  flag_time_counts = (
      flag_activity['time_only']
        .value_counts()
        .sort_values(ascending=False)
  )

  # Create full day time index at 15-min intervals
  full_day = pd.date_range("00:00", "23:45", freq="15min").time

  # Reindex your counts to include all times (fill missing with 0)
  flag_time_counts = flag_time_counts.reindex(full_day, fill_value=0)

  flag_counts_weekend=(
      df.groupby('Is_Weekend')['Demand_Response_Flag']
        .value_counts()
        .unstack(fill_value=0)
  )
  # group by weekday (0=Monday, 6=Sunday) and count
  flag_counts_weekday_total = (
      flag_activity.groupby(flag_activity.index.weekday)['Demand_Response_Flag']
                  .count()
  )
  # optional: rename index to weekday names
  flag_counts_weekday_total.index = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']


  pretty_print(flag_time_counts, "Frequent Times")
  pretty_print(flag_counts_weekend, "Weekend Flag Counts")
  pretty_print(flag_counts_day.sample(10), "Daily Flag Counts")
  pretty_print(flag_counts_month, "Monthly Flag Counts")
  pretty_print(flag_counts_season, "Seasonal Flag Counts")


  # Ploting flag activity by time , for instance site_A on 2.45 had 28 flag activity in total
  plt.figure(figsize=(14,6))
  flag_time_counts.plot(kind='bar', color='steelblue')

  plt.title(f"{df_name} – Flag Activity by Time of Day", fontsize=14)
  plt.xlabel("Time of Day", fontsize=12)
  plt.ylabel("Flag Count", fontsize=12)
  plt.xticks(range(len(full_day)), [t.strftime("%H:%M") for t in full_day],
              rotation=90, fontsize=8)
  plt.grid(axis='y', linestyle='--', alpha=0.6)
  plt.tight_layout()
  plt.show()


  # Ploting flag activity by time , for instance site_A had 130 flag activity on monday for the year
  plt.figure(figsize=(14,6))
  flag_counts_weekday_total.plot(kind='bar', color='steelblue')

  plt.title(f"{df_name} – Flag Activity by Day of Week", fontsize=14)
  plt.xlabel("Day of Week", fontsize=12)
  plt.ylabel("Flag Count", fontsize=12)
  plt.xticks(ticks=range(7), labels=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'],
            rotation=90, fontsize=10)

  plt.grid(axis='y', linestyle='--', alpha=0.6)
  plt.tight_layout()
  plt.show()



In [57]:
sns.set_style("whitegrid")  # cleaner seaborn plots

def agg_your_data(df, df_name):
    df = make_time_numeric(df)
    df['Season'] = df['Month'].apply(get_season_au)

    # -------------------
    # Aggregations
    daily = df.resample('D').agg({
        'Building_Power_kW': ['min','mean','max'],
        'Dry_Bulb_Temperature_C': ['min','mean','max'],
        'Global_Horizontal_Radiation_W/m2': ['min','mean','max'],
    })

    monthly = df.resample('M').agg({
        'Building_Power_kW': ['min','mean','max'],
        'Dry_Bulb_Temperature_C': ['min','mean','max'],
        'Global_Horizontal_Radiation_W/m2': ['min','mean','max'],
        'Demand_Response_Flag': 'nunique'
    })

    seasonal = df.groupby('Season').agg({
        'Building_Power_kW': ['min','mean','max'],
        'Dry_Bulb_Temperature_C': ['min','mean','max'],
        'Global_Horizontal_Radiation_W/m2': ['min','mean','max'],
        'Demand_Response_Flag': 'nunique'
    })

    daily.columns = ['_'.join(col) for col in daily.columns]

    # -------------------
    # Print summaries with nicer formatting
    print("\n" + "="*60)
    print(f"{df_name} – DAILY SAMPLE (100 rows)")
    print("="*60)
    print(daily.sample(100).round(2))

    print("\n" + "="*60)
    print(f"{df_name} – MONTHLY AGGREGATES")
    print("="*60)
    print(monthly.round(2))

    print("\n" + "="*60)
    print(f"{df_name} – SEASONAL AGGREGATES")
    print("="*60)
    print(seasonal.round(2))

    # -------------------
    # Daily line plot
    plt.figure(figsize=(15, 5))
    plt.plot(daily.index, daily['Building_Power_kW_mean'], label='Daily Mean Power', linewidth=2)
    plt.plot(daily.index, daily['Building_Power_kW_max'], label='Daily Max Power', alpha=0.7, linestyle='--')
    plt.plot(daily.index, daily['Building_Power_kW_min'], label='Daily Min Power', alpha=0.7, linestyle='--')
    plt.title(f"{df_name} – Daily Power Usage", fontsize=16)
    plt.xlabel("Date", fontsize=12)
    plt.ylabel("Power (kW)", fontsize=12)
    plt.legend(frameon=False)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # -------------------
    # Average 24-hour profile
    hourly_profile = df.groupby('Hour')['Building_Power_kW'].mean()
    plt.figure(figsize=(10, 5))
    sns.lineplot(x=hourly_profile.index, y=hourly_profile.values, linewidth=2)
    plt.title(f"{df_name} – Average 24-hour Power Profile", fontsize=16)
    plt.xlabel("Hour of Day", fontsize=12)
    plt.ylabel("Average Power (kW)", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # -------------------
    # Weekday vs weekend profile
    weekday_profile = df[df['Is_Weekend'] == False].groupby('Hour')['Building_Power_kW'].mean()
    weekend_profile = df[df['Is_Weekend'] == True].groupby('Hour')['Building_Power_kW'].mean()

    plt.figure(figsize=(10, 5))
    plt.plot(weekday_profile.index, weekday_profile.values, label="Weekday", linewidth=2)
    plt.plot(weekend_profile.index, weekend_profile.values, label="Weekend", linewidth=2, linestyle="--")
    plt.title(f"{df_name} – Weekday vs Weekend Average 24-hour Profile", fontsize=16)
    plt.xlabel("Hour of Day", fontsize=12)
    plt.ylabel("Average Power (kW)", fontsize=12)
    plt.legend(frameon=False)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # -------------------
    # Hourly profile by weekday
    hourly_week = df.groupby(['Weekday', 'Hour'])['Building_Power_kW'].mean().reset_index()
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=hourly_week, x='Hour', y='Building_Power_kW', hue='Weekday', palette='tab10')
    plt.title(f"{df_name} – Average 24-hour Power Profile by Weekday", fontsize=16)
    plt.xlabel("Hour of Day", fontsize=12)
    plt.ylabel("Average Power (kW)", fontsize=12)
    plt.legend(title="Weekday", labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # -------------------
    # Hourly profile by month
    hourly_month = df.groupby(['Month', 'Hour'])['Building_Power_kW'].mean().reset_index()
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=hourly_month, x='Hour', y='Building_Power_kW', hue='Month', palette='tab20')
    plt.title(f"{df_name} – Average 24-hour Power Profile by Month", fontsize=16)
    plt.xlabel("Hour of Day", fontsize=12)
    plt.ylabel("Average Power (kW)", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # -------------------
    # Hourly profile by season
    hourly_season = df.groupby(['Season', 'Hour'])['Building_Power_kW'].mean().reset_index()
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=hourly_season, x='Hour', y='Building_Power_kW', hue='Season', palette='Set2')
    plt.title(f"{df_name} – Average 24-hour Power Profile by Season", fontsize=16)
    plt.xlabel("Hour of Day", fontsize=12)
    plt.ylabel("Average Power (kW)", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


In [58]:
dict={"Site_A":site_A_dataset,
      "Site_B":site_B_dataset,
      "Site_C":site_C_dataset,
      "Site_D":site_D_dataset,
      }

for key, value in dict.items():
  agg_your_data(value,key)
  flag_analysis(value,key)




Output hidden; open in https://colab.research.google.com to view.